In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
import pickle
import matplotlib.pyplot as plt
import networkx as nx
import os

In [3]:
df = pd.read_csv('/Users/jchang153/Documents/UCLA-CAM/GV KG/Data/Tweets/All/bigbabies/bigbaby_r_t_30_s.csv')

In [4]:
df_c = pd.read_csv('user_network_black.csv')

## Community Centrality

Trying to find centrality for each of the communities created by Charlie. But I don't have monthly centralities, I am just using df_c for now. I need to run monthly centrality later.

In [ ]:
start_month = pd.Timestamp('2020-03')
end_month = pd.Timestamp('2021-06')
all_months = pd.period_range(start_month, end_month, freq='M')

In [ ]:
for month in all_months:
    print(str(month))

2020-03
2020-04
2020-05
2020-06
2020-07
2020-08
2020-09
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
2021-04
2021-05
2021-06


In [637]:
path = '../charlie_communities_black/Users Community in triples_'

for month in tqdm(all_months):
    df_com = pd.read_csv(path + str(month).replace('-', '_') + '.csv')
    df_com['user_id'] = df_com['user_id'].apply(ast.literal_eval)
    df_com = df_com[df_com['user_id'].apply(lambda x: len(x) >= 100)]
    com_centrs = []
    for com in tqdm(df_com['user_id']):
        temp = {}
        for user in com:
            if user in list(df_c['user']):
                # print('here')
                temp[user] = df_c[df_c['user'] == user].iloc[0]['eigenvector centrality']
        com_centrs.append(temp)
    df_com['centralities'] = com_centrs
    df_com.to_csv(path + str(month).replace('-', '_') + '.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 57.28it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 28.03it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 47.60it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 51.93it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 36.74it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.58it/s]


## Running Louvain on my user network

In [ ]:
start_date = pd.Timestamp('2020-03-11')
end_date = pd.Timestamp('2021-06-16')
# omit 2021-06-17 because it's a thursday, and we want nice weeks bc 2020-03-11 starts on wednesday

biweekly_dates = pd.date_range(start=start_date, end=end_date, freq='2W-WED')
biweekly_dates = biweekly_dates.union([end_date])

In [642]:
import community

In [668]:
for date in tqdm(biweekly_dates[:-1]):
    with open(f'./usersby2w-black/relations_{str(date)[:10]}.pkl', 'rb') as f:
        triples = pickle.load(f)

    G = nx.Graph()
    for u, v, w in triples:
        G.add_edge(u, v, weight=w)

    partition = community.best_partition(G, weight='weight')
    community_data = {'community_id': [], 'users': [], 'user_count': []}

    for node, comm_id in partition.items():
        if comm_id not in community_data['community_id']:
            community_data['community_id'].append(comm_id)
            community_data['users'].append([node])
            community_data['user_count'].append(1)
        else:
            idx = community_data['community_id'].index(comm_id)
            community_data['users'][idx].append(node)
            community_data['user_count'][idx] += 1

    df_communities = pd.DataFrame(community_data)
    df_communities['avg eigenvector centrality'] = [np.mean(df_c[df_c['user'].isin(user_list)]['eigenvector centrality']) for user_list in list(df_communities['users'])]
    df_communities['avg weighted degree'] = [np.mean(df_c[df_c['user'].isin(user_list)]['weighted degree']) for user_list in list(df_communities['users'])]
    df_communities['avg followers'] = [np.mean(df_c[df_c['user'].isin(user_list)]['followers']) for user_list in list(df_communities['users'])]
    df_communities['avg sent'] = [np.mean(df_c[df_c['user'].isin(user_list)]['average sent']) for user_list in list(df_communities['users'])]
    df_communities['avg num tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num tweets']) for user_list in list(df_communities['users'])]
    df_communities['avg num strong tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num strong tweets']) for user_list in list(df_communities['users'])]
    df_communities['avg num negative tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num negative tweets']) for user_list in list(df_communities['users'])]

    df_communities.to_csv('./usersby2w-black/user_communities_' + str(date)[:10] + '.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:33<00:00,  1.01s/it]


In [669]:
df_asdf = pd.read_csv('./usersby2w-black/user_communities_2020-12-30.csv')

In [670]:
df_asdf

,community_id,users,user_count,avg eigenvector centrality,avg weighted degree,avg followers,avg sent,avg num tweets,avg num strong tweets,avg num negative tweets
0,0,"[8.620798671012946e+17, 1.0226466006596936e+18...",193,0.021730,566.209650,3929.994819,-0.068242,27.823834,9.253886,6.191710
1,1,"[1.3158682071739556e+18, 28874068.0, 15723895....",35,0.021018,532.599425,2495.457143,-0.096786,22.685714,8.485714,5.742857
2,2,"[1.0835734352660972e+18, 48899468.0, 7.4235446...",13,0.023010,589.383352,13288.076923,-0.050492,24.769231,8.000000,5.076923
3,3,"[199120695.0, 3385139536.0, 17739546.0, 231313...",8,0.062996,1642.187116,2463.125000,-0.099596,73.750000,37.750000,27.250000
4,4,"[473384312.0, 9.182730501596242e+17, 90344035....",9,0.035629,895.786097,854.555556,-0.102428,34.555556,9.666667,6.555556
5,5,"[420025937.0, 4786423813.0, 39100425.0, 9.7116...",19,0.039958,1011.343465,1982.000000,-0.012150,35.210526,12.842105,8.526316


In [675]:
[int(x) for x in ast.literal_eval(df_asdf.iloc[3]['users'])]

[199120695,
 3385139536,
 17739546,
 2313136440,
 1337092240813006848,
 109360451,
 41747307,
 1325838919045296128]

In [683]:
for i in df[df['user_id'] == 3385139536]['text']:
    print(i)

“the police song” is arranged, produced, and written by dan devon. and inspired by the blacklivesmatter movement.… https://t.co/o3n4zjh5wq
riots won’t bring george floyd back or make the communities any safer or better. we’re living in the inevitable. there will be more blood.
buying into the media’s narrative that the georgefloyd tragedy is a black vs white thing is just as bad for you as… https://t.co/t4fcm7gyil
@lanayalewis now this was a racially motivated crime. i’m still not convinced about breanna taylor and george floyd tragedies.
@lanayalewis now this was a racially motivated crime. i’m still not convinced that breonna taylor and george floyd… https://t.co/xcyipabr4h
i get it but at the same time i don’t. why didn't the media or blacklivesmatter  highlight this tragedy?
i get it but at the same time i don’t. why didn't the media or blacklivesmatter   highlight this tragedy?… https://t.co/68pj5xlsp4
i hope joe biden doesn’t think he’s gonna win the black vote by promising us re

## Running Louvain on the thresholded user network

In [5]:
start_date = pd.Timestamp('2020-03-11')
end_date = pd.Timestamp('2021-06-16')
# omit 2021-06-17 because it's a thursday, and we want nice weeks bc 2020-03-11 starts on wednesday

biweekly_dates = pd.date_range(start=start_date, end=end_date, freq='2W-WED')
biweekly_dates = biweekly_dates.union([end_date])

In [6]:
import community

In [8]:
for date in tqdm(biweekly_dates[:-1]):
    with open(f'./usersby2w-black-threshold/relations_{str(date)[:10]}.pkl', 'rb') as f:
        triples = pickle.load(f)

    G = nx.Graph()
    for u, v, w in triples:
        if w >= 0.1:
            G.add_edge(u, v, weight=w)

    partition = community.best_partition(G, weight='weight')
    community_data = {'community_id': [], 'users': [], 'user_count': []}

    for node, comm_id in partition.items():
        if comm_id not in community_data['community_id']:
            community_data['community_id'].append(comm_id)
            community_data['users'].append([node])
            community_data['user_count'].append(1)
        else:
            idx = community_data['community_id'].index(comm_id)
            community_data['users'][idx].append(node)
            community_data['user_count'][idx] += 1

    df_communities = pd.DataFrame(community_data)
    df_communities['avg eigenvector centrality'] = [np.mean(df_c[df_c['user'].isin(user_list)]['eigenvector centrality']) for user_list in list(df_communities['users'])]
    df_communities['avg weighted degree'] = [np.mean(df_c[df_c['user'].isin(user_list)]['weighted degree']) for user_list in list(df_communities['users'])]
    df_communities['avg followers'] = [np.mean(df_c[df_c['user'].isin(user_list)]['followers']) for user_list in list(df_communities['users'])]
    df_communities['avg sent'] = [np.mean(df_c[df_c['user'].isin(user_list)]['average sent']) for user_list in list(df_communities['users'])]
    # df_communities['avg num tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num tweets']) for user_list in list(df_communities['users'])]
    # df_communities['avg num strong tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num strong tweets']) for user_list in list(df_communities['users'])]
    # df_communities['avg num negative tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num negative tweets']) for user_list in list(df_communities['users'])]

    df_communities.to_csv('./usersby2w-black-threshold/user_communities_' + str(date)[:10] + '.csv', index=False)

  0%|                                                                                                                            | 0/33 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: './usersby2w-black-threshold/relations_2020-03-11.pkl'

## Running Louvain on my user network all

In [4]:
df = pd.read_csv('/Users/jchang153/Documents/UCLA-CAM/GV KG/Data/Tweets/All/bigbabies/bigbaby_r_t_30_s.csv')

In [5]:
df_c = pd.read_csv('./user networks/user_network_all.csv')

In [6]:
df_c.columns

Index(['user', 'eigenvector centrality', 'weighted degree', 'num tweets',
       'followers', 'screen name', 'average sent', 'num strong tweets',
       'num negative tweets', 'prop strong tweets', 'prop negative tweets',
       'prod', 'logprod'],
      dtype='object')

In [7]:
start_date = pd.Timestamp('2020-03-11')
end_date = pd.Timestamp('2021-06-16')
# omit 2021-06-17 because it's a thursday, and we want nice weeks bc 2020-03-11 starts on wednesday

biweekly_dates = pd.date_range(start=start_date, end=end_date, freq='2W-WED')
biweekly_dates = biweekly_dates.union([end_date])

In [8]:
import community

In [9]:
for date in tqdm(biweekly_dates[:-1]):
    with open(f'./usersby2w-all/relations_{str(date)[:10]}.pkl', 'rb') as f:
        triples = pickle.load(f)

    G = nx.Graph()
    for u, v, w in triples:
        G.add_edge(u, v, weight=w)

    partition = community.best_partition(G, weight='weight')
    community_data = {'community_id': [], 'users': [], 'user_count': []}

    for node, comm_id in partition.items():
        if comm_id not in community_data['community_id']:
            community_data['community_id'].append(comm_id)
            community_data['users'].append([node])
            community_data['user_count'].append(1)
        else:
            idx = community_data['community_id'].index(comm_id)
            community_data['users'][idx].append(node)
            community_data['user_count'][idx] += 1

    df_communities = pd.DataFrame(community_data)
    df_communities['avg eigenvector centrality'] = [np.mean(df_c[df_c['user'].isin(user_list)]['eigenvector centrality']) for user_list in list(df_communities['users'])]
    df_communities['avg weighted degree'] = [np.mean(df_c[df_c['user'].isin(user_list)]['weighted degree']) for user_list in list(df_communities['users'])]
    df_communities['avg followers'] = [np.mean(df_c[df_c['user'].isin(user_list)]['followers']) for user_list in list(df_communities['users'])]
    df_communities['avg sent'] = [np.mean(df_c[df_c['user'].isin(user_list)]['average sent']) for user_list in list(df_communities['users'])]
    df_communities['avg num tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num tweets']) for user_list in list(df_communities['users'])]
    df_communities['avg num strong tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num strong tweets']) for user_list in list(df_communities['users'])]
    df_communities['avg num negative tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['num negative tweets']) for user_list in list(df_communities['users'])]
    df_communities['avg prop strong tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['prop strong tweets']) for user_list in list(df_communities['users'])]
    df_communities['avg prop negative tweets'] = [np.mean(df_c[df_c['user'].isin(user_list)]['prop negative tweets']) for user_list in list(df_communities['users'])]

    df_communities.to_csv('./usersby2w-all/user_communities_' + str(date)[:10] + '.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:36<00:00,  1.10s/it]
